In [11]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd


path = "/content/drive/My Drive/fire_predict/base_data/"
df_train = pd.read_csv(path + "PJT002_train.csv")
df_val = pd.read_csv(path + "PJT002_validation.csv")
df_test = pd.read_csv(path + "PJT002_test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import math
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm


def f_data(df_train, df_val, df_test):

    X_train = df_train.drop(['fr_yn'], 1)
    y_train = df_train['fr_yn']
    X_val = df_val.drop(['fr_yn'], 1)
    y_val = df_val['fr_yn']
    X_test = df_test

    df_all = pd.concat([X_train, X_val, X_test])

    categorical_cols = df_all.select_dtypes(['object']).columns
    for col in categorical_cols:
        df_all[col] = pd.Categorical(df_all[col]).codes

    X_train = df_all[:len(df_train)]
    X_val = df_all[len(df_train):-len(df_test)]
    X_test = df_all[-len(df_test):]

    # X_train = X_train.fillna(-1)
    # X_val = X_val.fillna(-1)
    # X_test = X_test.fillna(-1)

    return X_train, y_train, X_val, y_val, X_test

def f1_rfc(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

binary_y = {'N': 0, 'Y': 1}

df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

df_train_2 = df_train
df_val_2 = df_val
df_test_2 = df_test

df_train_2 = df_train_2.drop("id", axis=1)
df_val_2 = df_val_2.drop("id", axis=1)
df_test_2 = df_test_2.drop("id", axis=1)

df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
df_test_2 = df_test_2.drop("dt_of_fr", axis=1)


def year(x):
    if x != "NaN":
        if isinstance(x, float):
            if not math.isnan(x):
                if int(x) >10000000:
                    return x // 10000
                elif int(x) > 1000000:
                    return x // 1000
                elif int(x) > 100000:
                    return x // 100
                elif int(x) > 10000:
                    return x // 10
                elif int(x) > 1000:
                    return x
        elif isinstance(x, str):
            return int(x[:4])




df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

def population(x):
    if x < 1000:
        return "1k"
    elif x < 10000:
        return "10k"
    elif x < 100000:
        return "100k"
    elif x < 1000000:
        return "1000k"

df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [4]:
!pip install missingpy

In [5]:
s = df_train.isnull().sum()
c_list = np.where(s >= max(s)*0.5)
c_list[0]

array([ 13, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 172, 173, 174, 175, 176, 177, 178, 179])

In [6]:
s.index[c_list[0]]

Index(['prcpttn', 'lw_13101010', 'lw_13101110', 'lw_13101210', 'lw_13101211',
       'lw_13101310', 'lw_13101410', 'lw_13111010', 'lw_13111110',
       'lw_13121010', 'lw_13121011', 'lw_13131010', 'lw_13131110',
       'lw_13141010', 'lw_13141011', 'trgt_crtr',
       'fr_fghtng_fclt_spcl_css_5_yn', 'fr_fghtng_fclt_spcl_css_6_yn', 'us_yn',
       'dngrs_thng_yn', 'slf_fr_brgd_yn', 'blk_dngrs_thng_mnfctr_yn',
       'cltrl_hrtg_yn'],
      dtype='object')

In [0]:
df_2 = df_train.drop(list(s.index[c_list[0]]), axis=1)

In [8]:
df_2

,dt_of_fr,fr_yn,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,wnd_spd,wnd_drctn,hmdt,gas_engry_us_201401,ele_engry_us_201401,gas_engry_us_201402,ele_engry_us_201402,gas_engry_us_201403,ele_engry_us_201403,gas_engry_us_201404,ele_engry_us_201404,gas_engry_us_201405,ele_engry_us_201405,gas_engry_us_201406,ele_engry_us_201406,gas_engry_us_201407,ele_engry_us_201407,gas_engry_us_201408,ele_engry_us_201408,gas_engry_us_201409,ele_engry_us_201409,gas_engry_us_201410,ele_engry_us_201410,gas_engry_us_201411,ele_engry_us_201411,gas_engry_us_201412,ele_engry_us_201412,...,ele_engry_us_201803,gas_engry_us_201804,ele_engry_us_201804,gas_engry_us_201805,ele_engry_us_201805,gas_engry_us_201806,ele_engry_us_201806,gas_engry_us_201807,ele_engry_us_201807,gas_engry_us_201808,ele_engry_us_201808,gas_engry_us_201809,ele_engry_us_201809,gas_engry_us_201810,ele_engry_us_201810,gas_engry_us_201811,ele_engry_us_201811,gas_engry_us_201812,ele_engry_us_201812,jmk,id,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m
0,2017-10-20 05:54:00,1,단독주택,블록구조,3,69.420,69.4200,0.0,1977,1.0,0.0,주거용,10.7,0.9,200.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,대,106536,자연녹지지역,지정되지않음,단독,세로한면(불),경상남도 창녕군 창녕읍,17360.0,4547,NaN,133,137.0,N,112,0,0,1891,29231,11322,88,0
1,2018-09-30 08:26:15,0,NaN,NaN,3,46.290,46.2900,0.0,NaN,1.0,0.0,NaN,19.5,0.5,20.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대,86366,계획관리지역,지정되지않음,단독,세로한면(가),경상남도 의령군 가례면,1791.0,6388,122581.0,489,85.0,N,818,0,0,4533,20480,3369,7727,0
2,2016-10-30 14:57:00,1,공동주택,철근콘크리트구조,1,583.800,2516.7600,1446.0,2.00011e+07,5.0,0.0,주거용,15.5,2.0,90.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,답,107560,제2종일반주거지역,지정되지않음,연립,중로한면,경상남도 창녕군 창녕읍,17285.0,3340,618105.0,143,137.0,N,165,0,0,277,31197,12451,72,14
3,2016-06-14 05:23:58,0,단독주택,일반목구조,2,48.920,48.9200,0.0,1936,1.0,0.0,주거용,20.6,0.4,0.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,대,97550,제1종일반주거지역,지정되지않음,단독,세로한면(불),경상남도 창원시의창구 대산면,7327.0,179,719542.0,1585,176.0,N,131,0,0,438,4270,407,508,11
4,2018-04-22 05:38:59,0,NaN,NaN,2,0.000,0.0000,0.0,NaN,NaN,NaN,NaN,12.7,0.6,360.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,임,130501,자연환경보전지역,지정되지않음,자연림,세로한면(불),경상남도 창녕군 창녕읍,17278.0,4822,NaN,603,137.0,N,686,0,0,1702,29778,12487,707,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,2016-12-03 14:25:35,0,동.식물 관련시설,경량철골구조,2,423.000,423.0000,3010.0,1.99612e+07,NaN,NaN,NaN,11.2,0.2,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,목,91817,계획관리지역,지정되지않음,주거기타,세로한면(가),경상남도 함안군 칠원읍,20955.0,3435,NaN,895,139.0,N,1131,0,0,3591,3624,6679,914,0
59195,2016-10-29 10:10:00,0,NaN,NaN,35,5593.711,54835.0382,0.0,NaN,171.0,0.0,NaN,14.8,1.9,20.0,

In [0]:
categorical_cols = df_2.select_dtypes(['object']).columns
for col in categorical_cols:
    df_2[col] = pd.Categorical(df_2[col]).codes


In [10]:
s_2 = df_2[(df_2 < -1)].notnull().sum()
c_list_2 = np.where(s_2 >0)
s_2.index[c_list_2]

Index(['tmprtr', 'gas_engry_us_201507'], dtype='object')

In [11]:
df_2.tmprtr[df_2.tmprtr < 0]

9       -2.8
11      -3.1
25      -4.0
49      -1.5
59      -2.9
        ... 
59135   -0.9
59153   -5.9
59172   -0.6
59180   -8.6
59182   -7.7
Name: tmprtr, Length: 5146, dtype: float64

In [12]:
df_2.gas_engry_us_201507[df_2.gas_engry_us_201507 < 0]

21155   -11100.0
Name: gas_engry_us_201507, dtype: float64

In [0]:
df_2.tmprtr = df_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_2.gas_engry_us_201507 = df_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

In [0]:
df_2 = df_2.replace({-1:np.NaN})

In [15]:
(df_2.values < 0).sum().sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.


0

In [16]:
df_2 = df_2.drop(df_2.iloc[:,16:136].columns, axis=1)
df_2

,dt_of_fr,fr_yn,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,wnd_spd,wnd_drctn,hmdt,jmk,id,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m
0,45064,1,9.0,8.0,3,69.420,69.4200,0.0,100.0,1.0,0.0,6.0,283.85,0.9,200.0,96.0,4.0,106536,13.0,18.0,11.0,7.0,179.0,17360.0,4547,NaN,133,137.0,0.0,112,0,0,1891,29231,11322,88,0
1,56139,0,NaN,NaN,3,46.290,46.2900,0.0,NaN,1.0,0.0,NaN,292.65,0.5,20.0,74.0,4.0,86366,1.0,18.0,11.0,6.0,121.0,1791.0,6388,122581.0,489,85.0,0.0,818,0,0,4533,20480,3369,7727,0
2,33576,1,1.0,15.0,1,583.800,2516.7600,1446.0,6493.0,5.0,0.0,6.0,288.65,2.0,90.0,21.0,3.0,107560,17.0,18.0,24.0,11.0,179.0,17285.0,3340,618105.0,143,137.0,0.0,165,0,0,277,31197,12451,72,14
3,29293,0,9.0,10.0,2,48.920,48.9200,0.0,59.0,1.0,0.0,6.0,293.75,0.4,0.0,91.0,4.0,97550,15.0,18.0,11.0,7.0,241.0,7327.0,179,719542.0,1585,176.0,0.0,131,0,0,438,4270,407,508,11
4,51147,0,NaN,NaN,2,0.000,0.0000,0.0,NaN,NaN,NaN,NaN,285.85,0.6,360.0,89.0,13.0,130501,14.0,18.0,30.0,7.0,179.0,17278.0,4822,NaN,603,137.0,0.0,686,0,0,1702,29778,12487,707,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,34709,0,10.0,1.0,2,423.000,423.0000,3010.0,5501.0,NaN,NaN,NaN,284.35,0.2,0.0,41.0,6.0,91817,1.0,18.0,34.0,6.0,311.0,20955.0,3435,NaN,895,139.0,0.0,1131,0,0,3591,3624,6679,914,0
59195,33543,0,NaN,NaN,35,5593.711,54835.0382,0.0,NaN,171.0,0.0,NaN,287.95,1.9,20.0,72.0,4.0,99679,1.0,18.0,21.0,9.0,311.0,21014.0,1137,1849455.0,88,139.0,0.0,108,0,0,2629,77,1285,131,0
59196,49522,0,NaN,NaN,2,2436.820,2468.8000,6512.0,NaN,4.0,0.0,NaN,286.45,3.2,230.0,58.0,15.0,122377,1.0,18.0,5.0,11.0,306.0,3014.0,5875,NaN,25,139.0,0.0,968,2,0,4271,6139,4878,790,0
59197,44378,0,27.0,1.0,2,283.200,207.7100,0.0,7742.0,2.0,0.0,5.0,291.35,1.4,360.0,95.0,3.0,137293,4.0,18.0,13.0,3.0,305.0,3753.0,8579,NaN,510,139.0,0.0,177,0,0,1387,10142,3572,613,0


In [17]:
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler


# X is the complete data matrix
# X_incomplete has the same values as X except a subset have been replace with NaN

# Use 3 nearest rows which have a feature to fill in each row's missing features


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


In [18]:
X_filled_knn = KNN(k=3).fit_transform(df_2.values[::10])

Imputing row 1/5920 with 1 missing, elapsed time: 10.387
Imputing row 101/5920 with 0 missing, elapsed time: 10.404
Imputing row 201/5920 with 0 missing, elapsed time: 10.424
Imputing row 301/5920 with 0 missing, elapsed time: 10.443
Imputing row 401/5920 with 4 missing, elapsed time: 10.464
Imputing row 501/5920 with 7 missing, elapsed time: 10.479
Imputing row 601/5920 with 1 missing, elapsed time: 10.496
Imputing row 701/5920 with 4 missing, elapsed time: 10.515
Imputing row 801/5920 with 4 missing, elapsed time: 10.533
Imputing row 901/5920 with 4 missing, elapsed time: 10.550
Imputing row 1001/5920 with 6 missing, elapsed time: 10.570
Imputing row 1101/5920 with 3 missing, elapsed time: 10.592
Imputing row 1201/5920 with 4 missing, elapsed time: 10.612
Imputing row 1301/5920 with 7 missing, elapsed time: 10.631
Imputing row 1401/5920 with 8 missing, elapsed time: 10.651
Imputing row 1501/5920 with 4 missing, elapsed time: 10.666
Imputing row 1601/5920 with 6 missing, elapsed time:

In [0]:
X_filled_nnm = NuclearNormMinimization().fit_transform(df_2.values[::10])

----------------------------------------------------------------------------
	SCS v2.1.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 19246905
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 18183983, constraints m = 18622063
Cones:	primal zero / dual free vars: 219040
	linear vars: 657120
	sd vars: 17745903, sd blks: 1


In [1]:
X_incomplete_normalized = BiScaler().fit_transform(df_2.values[::10])
X_filled_softimpute = SoftImpute().fit_transform(X_incomplete_normalized)

NameError: ignored

In [0]:
nnm_mse = ((X_filled_nnm[missing_mask] - X[missing_mask]) ** 2).mean()
print("Nuclear norm minimization MSE: %f" % nnm_mse)

softImpute_mse = ((X_filled_softimpute[missing_mask] - X[missing_mask]) ** 2).mean()
print("SoftImpute MSE: %f" % softImpute_mse)

knn_mse = ((X_filled_knn[missing_mask] - X[missing_mask]) ** 2).mean()
print("knnImpute MSE: %f" % knn_mse)